<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_3_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC_%E1%84%80%E1%85%A1%E1%84%8B%E1%85%B5%E1%84%83%E1%85%B3_%E1%84%87%E1%85%A9%E1%86%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 여행 가이드 봇 만들기 - 혼자 실습

## RAG 실습 준비

In [ ]:
!pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 76.3 MB/s eta 0:00:00
 

In [ ]:
import os

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# ==========================================
# 0. 초기 설정
# ==========================================
import os
from getpass import getpass

# OpenAI API key
openai_api_key = getpass("Enter your OpenAI API key: ")

# API키 설정
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API key: ··········


In [ ]:
# 실습용 데이터 (제주도 맛집 가이드)
jeju_content = """
[제주도 맛집 & 여행 가이드]
1. 흑돼지 거리: 제주시 건입동에 위치. 저녁 7시 이후에는 웨이팅이 1시간 이상 발생할 수 있음.
   추천 메뉴는 흑돼지 오겹살(200g 2만원). 멜젓에 찍어 먹는 것이 특징.

2. 우도 땅콩 아이스크림: 우도 검멀레 해변 앞이 원조. 가격은 5,000원.
   고소한 땅콩 가루가 듬뿍 뿌려져 있음. 배 시간 때문에 오후 4시 이전에 가는 것을 추천.

3. 성산일출봉: 입장료 5,000원. 왕복 소요 시간 50분.
   매월 첫째 주 월요일은 휴관. 새벽 일출을 보려면 전날 근처 숙소 예약 필수.
"""

# 파일 생성
with open("jeju_guide.txt", "w", encoding="utf-8") as f:
    f.write(jeju_content)
print("제주도 가이드북 파일 생성 완료")

제주도 가이드북 파일 생성 완료


In [ ]:
# ==========================================
# 문서를 불러오고(Load) 자르기(Split)
# ==========================================
loader = TextLoader("jeju_guide.txt", encoding="utf-8")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

print(f"   -> 생성된 청크 개수: {len(splits)}개")

   -> 생성된 청크 개수: 4개


In [ ]:
# ==========================================
# 벡터 저장소를 만들기
# ==========================================
print("\n 벡터 DB 저장 중...")

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)


 벡터 DB 저장 중...


In [ ]:
# ==========================================
# RAG 체인을 완성
# ==========================================
print("\n 답변 생성 준비 중...")

retriever = vectorstore.as_retriever()

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트 템플릿
template = """
당신은 제주도 여행 가이드입니다.
[정보]를 바탕으로 여행객의 질문에 답변해주세요.

[정보]
{context}

[질문]: {question}
"""
prompt = PromptTemplate.from_template(template)


 답변 생성 준비 중...


In [ ]:
# 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# ==========================================
# 테스트 실행
# ==========================================
# 질문하기
query = "흑돼지 먹으려면 어디로 가야 해? 그리고 성산일출봉 입장료는 얼마야?"
print(f"\n 질문: {query}")

# 답변 출력
response = rag_chain.invoke(query)
print("\n=== AI 가이드의 답변 ===")
print(response)


 질문: 흑돼지 먹으려면 어디로 가야 해? 그리고 성산일출봉 입장료는 얼마야?

=== AI 가이드의 답변 ===
흑돼지를 먹으려면 제주시 건입동에 위치한 흑돼지 거리를 추천합니다. 저녁 7시 이후에는 웨이팅이 1시간 이상 발생할 수 있으니, 미리 가는 것이 좋습니다. 

성산일출봉의 입장료는 5,000원입니다. 왕복 소요 시간은 약 50분이며, 매월 첫째 주 월요일은 휴관하니 방문 계획에 참고하세요.


In [ ]:
# 파일 정리
os.remove("jeju_guide.txt")